In [1]:
import json
import csv
import random
import time
import torch
import torch.nn as nn
import numpy as np

In [2]:
class Baseline(nn.Module):
    def __init__(self, in_size = 512, out_size = 7):
        super(Baseline, self).__init__()

        self.W = nn.Linear(in_size, 7)
        self.out = nn.LogSoftmax(2)
        
    def forward(self, x):
        x = self.W(x)
        return self.out(x)
net = torch.load('models/USE-linear')
net.eval()

Baseline(
  (W): Linear(in_features=512, out_features=7, bias=True)
  (out): LogSoftmax()
)

In [80]:
use_en_vecs_t = np.load('data/snips_processed/USE-en-train.npy')
w2v_sv_vecs_t = np.load('data/snips_processed/W2V-sv-train.npy')
labs = np.load('data/snips_processed/train-labs.npy')

w2v_sv_vecs = np.squeeze(w2v_sv_vecs_t, axis=1)
use_en_vecs = np.squeeze(use_en_vecs_t, axis=1)

#appendd bias
w2v_sv_vecs = np.hstack([w2v_sv_vecs, np.array([np.ones(len(w2v_sv_vecs))]).T])

en_train = use_en_vecs[: 4000]
en_val = use_en_vecs[4000:8000]
en_test = use_en_vecs[8000:]

sv_train = w2v_sv_vecs[: 4000]
sv_val = w2v_sv_vecs[4000:8000]
sv_test = w2v_sv_vecs[8000:]

labs_train = labs[: 4000]
labs_val = labs[4000:8000]
labs_test = labs[8000:]
use_en_vecs.shape, w2v_sv_vecs.shape

((10000, 512), (10000, 301))

In [62]:
np.linalg.matrix_rank(w2v_sv_vecs), len(list(np.sum(w2v_sv_vecs, axis = 1))),len(set(np.sum(w2v_sv_vecs, axis = 1)))

(77, 10000, 9837)

In [63]:
# a x = b

x, res, rank, sing = np.linalg.lstsq(sv_train, en_train)

/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
x.shape,\
sv_train.shape,\
en_train.shape,\
(sv_train@x).shape,\
np.linalg.norm(en_train -(sv_train@x)),\
np.linalg.norm(en_val -(sv_val@x))

((301, 512),
 (4000, 301),
 (4000, 512),
 (4000, 512),
 48.85690431195968,
 59419169996.74705)

In [94]:
def eval(model, vectors, labels):
    with torch.no_grad():
        vectors = list(map(lambda x: np.array([x]) , vectors))
        #labels = list(map(lambda x: np.array([x]) , labels))
        vectors = torch.tensor(list(vectors)).float()
        labels = torch.tensor(labels)
            
        model_out = model.forward(vectors)
        right = 0
        
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1
        return right/len(model_out)
eval(net,en_train, labs_train), \
eval(net, sv_train@x, labs_train),\
eval(net, sv_val@x, labs_val),\
eval(net, sv_test@x, labs_test)

(0.96125, 0.7325, 0.71475, 0.701)